In [1]:
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopy

In [2]:
#lists tables and indexes in the database
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM sqlite_master; "

master_sqlite = pd.read_sql(query, db)

db.close()

master_sqlite

,type,name,tbl_name,rootpage,sql
0,table,hop_team,hop_team,2,"CREATE TABLE ""hop_team"" (\n""from_npi"" INTEGER,..."
1,table,cbsa,cbsa,2876549,"CREATE TABLE ""cbsa"" (\n""zip"" INTEGER,\n ""cbsa..."
2,table,taxonomy,taxonomy,2877873,"CREATE TABLE ""taxonomy"" (\n""NPI"" INTEGER,\n ""..."
3,table,nucc,nucc,3298774,"CREATE TABLE ""nucc"" (\n""code"" TEXT,\n ""groupi..."
4,index,zip,cbsa,3298917,CREATE INDEX zip ON cbsa(zip)
5,index,NPI,taxonomy,3299319,CREATE INDEX NPI ON taxonomy(NPI)
6,index,code,nucc,3666969,CREATE INDEX code ON nucc(code)
7,table,nppes,nppes,3666974,"CREATE TABLE ""nppes"" (\n""npi"" INTEGER,\n ""ent..."
8,index,nppes_npi,nppes,3810771,CREATE INDEX nppes_npi ON nppes(npi)
9,index,from_npi,hop_team,2131546,CREATE INDEX from_npi ON hop_team(from_npi)


In [9]:
query = '''
SELECT * 
FROM hop_team
LIMIT 5'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    hop_team_5 = pd.read_sql(query,db)
db.close()
hop_team_5.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880


In [10]:
query = '''
SELECT * 
FROM cbsa
LIMIT 5'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    cbsa_5 = pd.read_sql(query,db)

db.close()
cbsa_5.head()

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,683,41900,SAN GERMAN,PR,0.999842,1.0,1.0,0.999855
1,683,32420,SAN GERMAN,PR,0.000158,0.0,0.0,0.000145
2,923,41980,SAN JUAN,PR,1.000000,1.0,1.0,1.000000
3,1010,44140,BRIMFIELD,MA,0.976896,1.0,1.0,0.977816
4,1010,49340,BRIMFIELD,MA,0.023104,0.0,0.0,0.022184


In [11]:
query = '''
SELECT * 
FROM taxonomy
LIMIT 5'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    taxonomy_5 = pd.read_sql(query,db)

db.close()
taxonomy_5.head()

,NPI,taxonomy_switch,taxonomy_code
0,1679576722,Y,207X00000X
1,1588667638,Y,207RC0000X
2,1497758544,Y,251G00000X
3,1306849450,None,None
4,1215930367,N,174400000X


In [12]:
query = '''
SELECT * 
FROM nucc
LIMIT 5'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    nucc_5 = pd.read_sql(query,db)

db.close()
nucc_5.head()

,code,grouping,classification,specialization,definition,notes,display_name,section
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,None,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,None,Clinical & Laboratory Immunology (Allergy & Im...,Individual


In [3]:
query = '''
WITH to_npi AS(
SELECT npi AS to_npi,
    "provider_organization_name_(legal_business_name)" AS to_org_name,
    "provider_last_name_(legal_name)" AS to_last_name,
    provider_first_name AS to_first_name,
    provider_middle_name AS to_middle_name,
    provider_credential_text AS to_credential,
    provider_first_line_business_practice_location_address AS to_address,
    provider_second_line_business_practice_location_address AS to_2_address,
    provider_business_practice_location_address_city_name AS to_city,
    provider_business_practice_location_address_state_name AS to_state,
    substring(nppes.provider_business_practice_location_address_postal_code, 1, 5) AS to_zip
FROM nppes
WHERE entity_type_code = 2.0
),
Nash_cbsa AS(
SELECT CAST(zip as TEXT) as zip,
cbsa,
usps_zip_pref_city,
usps_zip_pref_state,
res_ratio,
bus_ratio,
oth_ratio,
tot_ratio 
FROM cbsa 
WHERE cbsa = 34980
),
taxon AS (
SELECT npi, taxonomy_code
FROM taxonomy
WHERE taxonomy_switch = 'Y'
), 
codes AS(
SELECT code, classification, specialization, display_name
FROM nucc
)
SELECT *
FROM to_npi
JOIN Nash_cbsa
on to_npi.to_zip = zip
JOIN taxon
ON to_npi.to_npi = npi
JOIN codes
ON codes.code = taxon.taxonomy_code

'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    to_npi = pd.read_sql(query,db)

db.close()
to_npi.to_csv('data/to_npi.csv')
to_npi.head()

,to_npi,to_org_name,to_last_name,to_first_name,to_middle_name,to_credential,to_address,to_2_address,to_city,to_state,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,npi,taxonomy_code,code,classification,specialization,display_name
0,1881697092,"RHS, INC.",None,None,None,None,1330 TROTWOOD AVE,None,COLUMBIA,TN,...,0.993028,0.999619,1.0,0.993669,1881697092,332B00000X,332B00000X,Durable Medical Equipment & Medical Supplies,None,Durable Medical Equipment & Medical Supplies
1,1881697092,"RHS, INC.",None,None,None,None,1330 TROTWOOD AVE,None,COLUMBIA,TN,...,0.993028,0.999619,1.0,0.993669,1881697092,332B00000X,332B00000X,Durable Medical Equipment & Medical Supplies,None,Durable Medical Equipment & Medical Supplies
2,1881697092,"RHS, INC.",None,None,None,None,1330 TROTWOOD AVE,None,COLUMBIA,TN,...,0.993028,0.999619,1.0,0.993669,1881697092,332B00000X,332B00000X,Durable Medical Equipment & Medical Supplies,None,Durable Medical Equipment & Medical Supplies
3,1326041534,MURFREESBORO CONVALESCENT SERVICE,None,None,None,None,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,...,1.000000,1.000000,1.0,1.000000,1326041534,341600000X,341600000X,Ambulance,None,Ambulance
4,1326041534,MURFREESBORO CONVALESCENT SERVICE,None,None,None,None,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,...,1.000000,1.000000,1.0,1.000000,1326041534,341600000X,341600000X,Ambulance,None,Ambulance


In [6]:
to_npi = pd.read_csv('data/to_npi.csv')

In [4]:
to_npi.shape

(26250, 25)

Clean up columns
    Drop code or taxonomy_code,
    drop one of the Zip code fields. 
    drop npi as it's captured in to_npi
Merge hop_team with to_npi and from_npi
    clean up columns 


In [2]:
query = '''
WITH from_npi AS (
SELECT npi AS from_npi,
    "provider_organization_name_(legal_business_name)" AS from_org_name,
    "provider_last_name_(legal_name)" AS from_last_name,
    provider_first_name AS from_first_name,
    provider_middle_name AS from_middle_name,
    provider_credential_text AS from_credential,
    provider_first_line_business_practice_location_address AS from_address,
    provider_second_line_business_practice_location_address AS from_2_address,
    provider_business_practice_location_address_city_name AS from_city,
    provider_business_practice_location_address_state_name AS from_state,
    substring(nppes.provider_business_practice_location_address_postal_code, 1, 5) AS from_zip
FROM nppes
WHERE entity_type_code = 1.0
), 
surrounding_cbsa AS(
SELECT CAST(zip as TEXT) as zip,
cbsa,
usps_zip_pref_city,
usps_zip_pref_state,
res_ratio,
bus_ratio,
oth_ratio,
tot_ratio
FROM cbsa
WHERE cbsa = 17300 
OR cbsa = 14540 
OR cbsa = 27180 
OR cbsa = 18260 
OR cbsa = 30280 
OR cbsa = 43180 
OR cbsa = 46100 
OR cbsa = 32660 
OR cbsa = 16860
),
taxon AS (
SELECT npi, taxonomy_code
FROM taxonomy
WHERE taxonomy_switch = 'Y'
),
codes AS(
SELECT code, classification, specialization, display_name
FROM nucc
)
SELECT *
FROM from_npi
JOIN surrounding_cbsa
on from_npi.from_zip = zip
JOIN taxon
ON from_npi.from_npi = npi
JOIN codes
ON codes.code = taxon.taxonomy_code
'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    from_npi = pd.read_sql(query,db)

db.close()
from_npi.to_csv('data/from_npi.csv')
from_npi.head()

,from_npi,from_org_name,from_last_name,from_first_name,from_middle_name,from_credential,from_address,from_2_address,from_city,from_state,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,npi,taxonomy_code,code,classification,specialization,display_name
0,1003112277,None,SMITH,TABITHA,M,F.N.P.-C,89 MEARS DRIVE,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1003112277,363LF0000X,363LF0000X,Nurse Practitioner,Family,Family Nurse Practitioner
1,1003854258,None,YOUNG,KEITH,None,M.D.,205 S MCCRARY ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1003854258,207Q00000X,207Q00000X,Family Medicine,None,Family Medicine Physician
2,1013238856,None,PINKSTON,GAVIN,BUTLER,MD,205 S MCCRARY ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1013238856,207Q00000X,207Q00000X,Family Medicine,None,Family Medicine Physician
3,1063059772,None,APPLEGATE,MICAH,KATHERINE,None,119 W HIGH ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1063059772,225200000X,225200000X,Physical Therapy Assistant,None,Physical Therapy Assistant
4,1073583449,None,COOMES,BERNARD,JOHN,DC,313 W MAIN ST,None,WOODBURY,TN,...,0.009001,0.008565,0.0,0.008846,1073583449,111NI0900X,111NI0900X,Chiropractor,Internist,Internist Chiropractor


In [3]:
from_npi = pd.read_csv('data/from_npi.csv')
from_npi.head()

,Unnamed: 0,from_npi,from_org_name,from_last_name,from_first_name,from_middle_name,from_credential,from_address,from_2_address,from_city,...,res_ratio,bus_ratio,oth_ratio,tot_ratio,npi,taxonomy_code,code,classification,specialization,display_name
0,0,1003112277,NaN,SMITH,TABITHA,M,F.N.P.-C,89 MEARS DRIVE,NaN,WOODBURY,...,0.009001,0.008565,0.0,0.008846,1003112277,363LF0000X,363LF0000X,Nurse Practitioner,Family,Family Nurse Practitioner
1,1,1003854258,NaN,YOUNG,KEITH,NaN,M.D.,205 S MCCRARY ST,NaN,WOODBURY,...,0.009001,0.008565,0.0,0.008846,1003854258,207Q00000X,207Q00000X,Family Medicine,NaN,Family Medicine Physician
2,2,1013238856,NaN,PINKSTON,GAVIN,BUTLER,MD,205 S MCCRARY ST,NaN,WOODBURY,...,0.009001,0.008565,0.0,0.008846,1013238856,207Q00000X,207Q00000X,Family Medicine,NaN,Family Medicine Physician
3,3,1063059772,NaN,APPLEGATE,MICAH,KATHERINE,NaN,119 W HIGH ST,NaN,WOODBURY,...,0.009001,0.008565,0.0,0.008846,1063059772,225200000X,225200000X,Physical Therapy Assistant,NaN,Physical Therapy Assistant
4,4,1073583449,NaN,COOMES,BERNARD,JOHN,DC,313 W MAIN ST,NaN,WOODBURY,...,0.009001,0.008565,0.0,0.008846,1073583449,111NI0900X,111NI0900X,Chiropractor,Internist,Internist Chiropractor


(26250, 26)

In [11]:
nppes_5['zip'].str.slice(start = 0, stop = 5)

0    68847
1    32204
2    28304
3     None
4    77090
Name: zip, dtype: object

In [2]:
#filter hop_team for entity_type 1.0 or 2.0 and transaction_count>=50
#and average_day_wait<50 and taxonomy_switch = 'Y'
query = '''
WITH from_npi_list AS (
SELECT npi
FROM nppes
WHERE entity_type_code = 1.0
),
to_npi_list AS (
SELECT npi
FROM nppes
WHERE entity_type_code = 2.0
)
SELECT * 
FROM hop_team
WHERE from_npi IN from_npi_list 
AND to_npi IN to_npi_list
AND transaction_count >= 50 AND average_day_wait < 50

'''

with sqlite3.connect('data/hop_team.sqlite') as db:
    hop_team = pd.read_sql(query,db)
db.close()
hop_team.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1265564736,1003000209,22,148,0.000,0.000
1,1316131865,1003000217,47,52,27.212,50.908
2,1881710101,1003000217,185,227,0.000,0.000
3,1962528539,1003000217,308,412,0.000,0.000
4,1548451677,1003000274,64,76,35.803,42.841


In [4]:
query = '''
SELECT CAST(zip as TEXT) as zip,
cbsa,
usps_zip_pref_city,
usps_zip_pref_state,
res_ratio,
bus_ratio,
oth_ratio,
tot_ratio 
FROM cbsa 
WHERE cbsa = 34980
'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    nash_cbsa = pd.read_sql(query,db)
db.close()
nash_cbsa.head()

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37190,34980,WOODBURY,TN,0.990999,0.991435,1.0,0.991154
1,37046,34980,COLLEGE GROVE,TN,0.992654,1.000000,1.0,0.992839
2,37077,34980,HENDERSONVILLE,TN,1.000000,1.000000,1.0,1.000000
3,37101,34980,MC EWEN,TN,0.002121,0.000000,0.0,0.002000
4,37026,34980,BRADYVILLE,TN,0.967974,1.000000,1.0,0.969112


In [5]:
#SQL query to create CBSAs that are not the Nashville CBSA
query = '''
SELECT CAST(zip as TEXT) as zip,
cbsa,
usps_zip_pref_city,
usps_zip_pref_state,
res_ratio,
bus_ratio,
oth_ratio,
tot_ratio
FROM cbsa
WHERE cbsa = 17300 
OR cbsa = 14540 
OR cbsa = 27180 
OR cbsa = 18260 
OR cbsa = 30280 
OR cbsa = 43180 
OR cbsa = 46100 
OR cbsa = 32660 
OR cbsa = 16860
'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    NotNash_cbsa = pd.read_sql(query,db)
db.close()
NotNash_cbsa.head()

,zip,cbsa,usps_zip_pref_city,usps_zip_pref_state,res_ratio,bus_ratio,oth_ratio,tot_ratio
0,37190,32660,WOODBURY,TN,0.009001,0.008565,0.0,0.008846
1,38006,27180,BELLS,TN,0.876518,0.919463,1.0,0.879462
2,37338,16860,GRAYSVILLE,TN,0.312344,0.105263,0.0,0.305961
3,37414,16860,CHATTANOOGA,TN,1.000000,1.000000,1.0,1.000000
4,38358,27180,MILAN,TN,0.997636,1.000000,1.0,0.997941


In [6]:
query = '''
SELECT code, classification, specialization, display_name
FROM nucc
'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    nucc = pd.read_sql(query,db)
db.close()
nucc.head()

,code,classification,specialization,display_name
0,193200000X,Multi-Specialty,None,Multi-Specialty Group
1,193400000X,Single Specialty,None,Single Specialty Group
2,207K00000X,Allergy & Immunology,None,Allergy & Immunology Physician
3,207KA0200X,Allergy & Immunology,Allergy,Allergy Physician
4,207KI0005X,Allergy & Immunology,Clinical & Laboratory Immunology,Clinical & Laboratory Immunology (Allergy & Im...


In [8]:
query = '''
WITH nucc AS (
SELECT code, classification, specialization, display_name
FROM nucc
)
SELECT npi, taxonomy_code
FROM taxonomy
WHERE taxonomy_switch = 'Y'
LEFT JOIN nucc

'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    taxonomy = pd.read_sql(query,db)
db.close()
taxonomy.head()

,NPI,taxonomy_code
0,1679576722,207X00000X
1,1588667638,207RC0000X
2,1497758544,251G00000X
3,1023011178,251G00000X
4,1841293990,231H00000X


In [ ]:
hop_team.shape

In [ ]:
query = '''
WITH nucc AS (
SELECT code, classification, specialization, display_name
FROM nucc
),
hop_team AS(

),
nash_cbsa AS(

)



SELECT npi, taxonomy_code
FROM taxonomy
WHERE taxonomy_switch = 'Y'

'''
with sqlite3.connect('data/hop_team.sqlite') as db:
    full_set = pd.read_sql(query,db)
db.close()
full_set.head()